# Teste do projeto spot
    - pegar o token
    - extrair os dados da api
    - criar funcoes para extrair os dados da API
    - cria funcao para exportar para mongo

# URL's

    # query = barcelona
        # - 'https://api.foursquare.com/v3/places/search?query=barcelona' 
    # query = caffe - near = barcelona
        - 'https://api.foursquare.com/v3/places/search?query=coffee&near=barcelona' 
    # query = caffe / categories = 11124 Business and Professional Services > Office / near = barcelona
        - 'https://api.foursquare.com/v3/places/search?query=coffee&categories=11124&near=barcelona'     
    # categories = 11124 Business and Professional Services > Office / near = barcelona
        - 'https://api.foursquare.com/v3/places/search?categories=11124&near=barcelona'
    # near = barcelona
        - 'https://api.foursquare.com/v3/places/search?near=barcelona'
    # 12057	Community and Government > Education > Primary and Secondary School
        - https://api.foursquare.com/v3/places/search?query=escola&categories=12057&near=barcelona    
    # 12059	Community and Government > Education > Primary and Secondary School > High School (Tem perto do trablho)
    - https://api.foursquare.com/v3/places/search?query=escola&categories=12059&near=barcelona


# Livrarias

In [1]:
import requests
import json
import geopandas
import cartoframes
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

# API FourSquare 

# Token

In [2]:
load_dotenv()

True

In [3]:
token = getpass.getpass("token: ")


# Funcao Query

In [32]:
def extrator_query(query):
    url = f"https://api.foursquare.com/v3/places/search?query={query}"
                
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()
    return response

# Funcao Query Near

In [4]:
def extrator_ll(query,lat, long):
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": query,
        'll': f'{lat},{long}'
        }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.request("GET", url, params=params, headers=headers).json()
    return response

In [20]:
latlong = extrator_ll('school','41.4039759','2.1906131')
latlong

{'results': [{'fsq_id': '4ca4c2c2750ca093b9262467',
   'categories': [{'id': 12053,
     'name': 'Language School',
     'short_name': 'Language School',
     'plural_name': 'Language Schools',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'LikelyOpen',
   'distance': 3339,
   'geocodes': {'main': {'latitude': 41.376799, 'longitude': 2.173472},
    'roof': {'latitude': 41.376799, 'longitude': 2.173472}},
   'link': '/v3/places/4ca4c2c2750ca093b9262467',
   'location': {'address': 'Avenida Drassanes, 14',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'formatted_address': 'Avenida Drassanes, 14, 08001 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08001',
    'region': 'Catalunya'},
   'name': "Escola Oficial d'Idiomes",
   'related_places': {'children': [{'fsq_id': '4decc5f888774880e356257f',
      'categories': [{'id': 12053,
        'name': 'Language S

In [37]:
for result in latlong['results']:
    name = result['name']
    address = result['location']['formatted_address']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    print(f"Name: {name} - Address: {address} - Latitude: {latitude}, Longitude: {longitude}")

Name: Escola Oficial d'Idiomes - Address: Avenida Drassanes, 14, 08001 Barcelona Catalunya - Latitude: 41.376799, Longitude: 2.173472
Name: Escola Oficial d´Idiomes Barcelona Vall d´Hebron - Address: Avinguda del Jordà, 18, 08035 Barcelona Catalunya - Latitude: 41.4268, Longitude: 2.13849
Name: Agora Sant Cugat International School - Address: Ferrer I Guàrdia, s/n, 08174 Sant Cugat Catalunya - Latitude: 41.494532, Longitude: 2.077463
Name: Escola Oficial D. Idiomes de Sabadell - Address: Brujas, 165, 08202 Sabadell Catalunya - Latitude: 41.55325, Longitude: 2.112351
Name: IESE Business School - North Campus - Address: Arnus I Gari, 3, Barcelona Catalunya - Latitude: 41.393657, Longitude: 2.10537
Name: Centre D'educacio Infantil I Primaria Ramon Llull - Address: Avenida Diagonal, 275, 08013 Barcelona Catalunya - Latitude: 41.400852, Longitude: 2.17776
Name: Vila Olímpica - Address: Calle Carmen Amaya, 2, 08005 Barcelona Catalunya - Latitude: 41.39396, Longitude: 2.202743
Name: Maristes 

In [38]:
def locals(data):
    list_local = []
    df = pd.DataFrame(columns = ['Name', 'Address', 'Latitude', 'Longitude'])
    for result in data['results']:
        local = {}
        local['Name'] = result['name']
        local['Address'] = result['location']['formatted_address'] 
        local['Latitude'] = result['geocodes']['main']['latitude']
        local['Longitude'] = result['geocodes']['main']['longitude']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    
        

In [40]:
result = locals(latlong)
result

,Name,Address,Latitude,Longitude
0,Escola Oficial d'Idiomes,"Avenida Drassanes, 14, 08001 Barcelona Catalunya",41.376799,2.173472
1,Escola Oficial d´Idiomes Barcelona Vall d´Hebron,"Avinguda del Jordà, 18, 08035 Barcelona Catalunya",41.426800,2.138490
2,Agora Sant Cugat International School,"Ferrer I Guàrdia, s/n, 08174 Sant Cugat Catalunya",41.494532,2.077463
3,Escola Oficial D. Idiomes de Sabadell,"Brujas, 165, 08202 Sabadell Catalunya",41.553250,2.112351
4,IESE Business School - North Campus,"Arnus I Gari, 3, Barcelona Catalunya",41.393657,2.105370
5,Centre D'educacio Infantil I Primaria Ramon Llull,"Avenida Diagonal, 275, 08013 Barcelona Catalunya",41.400852,2.177760
6,Vila Olímpica,"Calle Carmen Amaya, 2, 08005 Barcelona Catalunya",41.393960,2.202743
7,Maristes la Immaculada,Barcelona Catalunya,41.399027,2.171938
8,Kiddys Music,"Roger de Flor, 291 (Sant Antoni Maria Claret),...",41.404544,2.164833
9,PRBB - Parc de Recerca Biomèdica de Barcelona,"Dr. Aiguader, 88, 08003 Barcelona Catalunya",41.385090,2.194550


In [49]:
test_map = Map(location=['41.4046268','2.1893276'], zoom_start=15)

In [54]:
for i, r in result.iterrows():
    name = r['Name']
    lat = r['Latitude']
    long = r['Longitude']
    color = 'blue'
    icon_color = 'black'
    icon = 'graduation-cap'
    icon = Icon(
                color = color,
                icon_color = icon_color,
                icon = icon,
                prefix="fa" # font-awesome website: fa
        )
    test_marker = Marker(location=[lat, long], tooltip=name, icon= icon)
    test_marker.add_to(test_map)
test_map

# Como enviar o json extraido da api para meu mongo

In [ ]:
def export_to_mongodb(data, col):
    # Conectar ao MongoDB (certifique-se de ter o servidor MongoDB em execução)
    client = MongoClient("localhost", 27017)

    # Escolher ou criar um banco de dados e uma coleção no MongoDB
    db = client["Project_Spot"]
    collection = db[col]

    # Inserir os dados no MongoDB
    result = collection.insert_one(data)

    # Imprimir o ID do documento inserido
    print(f"Documento inserido com ID: {result.inserted_id}")

    return result

In [ ]:
# Enviar dados extraídos para o MongoDB
export_to_mongodb(api_data, 'coffe shop')

Documento inserido com ID: 65c69c379bac309e38f65eb3


InsertOneResult(ObjectId('65c69c379bac309e38f65eb3'), acknowledged=True)

In [ ]:
# verificar coordenadas
for result in api_data['results']:
    geocodes = result.get('geocodes', {})  # Get the 'geocodes' dictionary, default to an empty dictionary if not present
    for key, coordinates in geocodes.items():
        print(f" coordinates: {coordinates['latitude']}, {coordinates['longitude']}")




 coordinates: 41.384965, 2.164582
 coordinates: 41.384965, 2.164582
 coordinates: 41.381567, 2.159095
 coordinates: 41.381567, 2.159095
 coordinates: 41.398077, 2.162116
 coordinates: 41.398077, 2.162116
 coordinates: 41.400852, 2.17776
 coordinates: 41.400852, 2.17776
 coordinates: 41.376394, 2.160389
 coordinates: 41.376394, 2.160389
 coordinates: 41.373088, 2.174453
 coordinates: 41.373088, 2.174453
 coordinates: 41.402755, 2.183019
 coordinates: 41.402755, 2.183019
 coordinates: 41.391011, 2.147691
 coordinates: 41.391011, 2.147691
 coordinates: 41.40671, 2.155362
 coordinates: 41.407049, 2.155141
 coordinates: 41.407049, 2.155141
 coordinates: 41.39396, 2.202743
 coordinates: 41.39396, 2.202743
